In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, rnn_cell='lstm', dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()  # Initialize the parent class.
        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size)  # Create an embedding layer.
        self.dropout = nn.Dropout(dropout)  # Create a dropout layer.
        self.hidden_size = hidden_size  # Store the hidden size.
        self.num_layers = num_layers  # Store the number of layers.
        self.bidirectional = bidirectional  # Store whether the RNN is bidirectional.
        
        rnn_hidden_size = hidden_size // 2 if bidirectional else hidden_size  # Adjust hidden size for bidirectional RNN.
        
        if rnn_cell.lower() == 'lstm':
            self.rnn = nn.LSTM(embedding_size, rnn_hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout), bidirectional=bidirectional)  # Create an LSTM layer.
        elif rnn_cell.lower() == 'gru':
            self.rnn = nn.GRU(embedding_size, rnn_hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout), bidirectional=bidirectional)  # Create a GRU layer.
        else:
            self.rnn = nn.RNN(embedding_size, rnn_hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout), bidirectional=bidirectional)  # Create an RNN layer.
    
    def forward(self, x):
        embedded = self.embedding(x)  # Embed the input sequences.
        embedded = self.dropout(embedded)  # Apply dropout to the embeddings.
        outputs, hidden = self.rnn(embedded)  # Pass the embeddings through the RNN.

        if self.bidirectional:  # If the RNN is bidirectional.
            if isinstance(hidden, tuple):  # If the hidden state is a tuple (LSTM case).
                h_n, c_n = hidden  # Unpack the hidden states (hidden and cell states for LSTM).
#                 print('enc h bef dir',h_n.shape)  
#                 print('enc c bef dir',c_n.shape)  
                h_n = torch.cat((h_n[0::2], h_n[1::2]), dim=2)  # Concatenate the forward and backward hidden states.
                c_n = torch.cat((c_n[0::2], c_n[1::2]), dim=2)  # Concatenate the forward and backward cell states.
#                 print('enc h af dir',h_n.shape) 
#                 print('enc c af dir',c_n.shape)  
                hidden = (h_n, c_n)  # Pack the adjusted hidden states back into a tuple.
            else:  # If the hidden state is not a tuple (GRU/RNN case).
#                 print('enc hidd bef dir',hidden.shape) 
                hidden = torch.cat((hidden[0::2], hidden[1::2]), dim=2)  # Concatenate the forward and backward hidden states.
#                 print('after dir enc:',hidden.shape) 

        return hidden  # Return the RNN hidden states.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, num_layers, encoder_num_layers, rnn_cell='lstm', dropout=0.5, bidirectional=True):
        super(Decoder, self).__init__()  # Initialize the parent class.
        self.embedding = nn.Embedding(num_embeddings=output_size, embedding_dim=embedding_size)  # Create an embedding layer.
        self.dropout = nn.Dropout(dropout)  # Create a dropout layer.
        self.output_size = output_size  # Store the output size.
        self.hidden_size = hidden_size * encoder_num_layers if bidirectional else hidden_size  # Adjust hidden size for bidirectional encoder.
        self.num_layers = num_layers  # Store the number of layers.
        
        if rnn_cell.lower() == 'lstm':
            self.rnn = nn.LSTM(embedding_size, self.hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout))  # Create an LSTM layer.
        elif rnn_cell.lower() == 'gru':
            self.rnn = nn.GRU(embedding_size, self.hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout))  # Create a GRU layer.
        else:
            self.rnn = nn.RNN(embedding_size, self.hidden_size, num_layers, batch_first=True, dropout=(0 if num_layers == 1 else dropout))  # Create an RNN layer.
        
        self.fc = nn.Linear(self.hidden_size, output_size)  # Create a fully connected layer for output.

    def forward(self, x, hidden):
        x = x.unsqueeze(1)  # Add a singleton dimension to the input tensor.
        embedded = self.dropout(self.embedding(x))  # Embed the input sequences and apply dropout.
        output, hidden = self.rnn(embedded, hidden)  # Pass the embedded input through the RNN.
        output = self.fc(self.dropout(output.squeeze(1)))  # Apply dropout and pass through the fully connected layer.
        return output, hidden  # Return the output and hidden states.

In [ ]:
class Seq_to_Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq_to_Seq, self).__init__()  # Initialize the parent class.
        self.encoder = encoder  # Store the encoder module.
        self.decoder = decoder  # Store the decoder module.
        
    def forward(self, source, target, teaching_force_ratio=0.5):
        batch_size = source.size(0)  # Get the batch size.
        target_len = target.size(1)  # Get the target sequence length.
        target_vocab_size = self.decoder.output_size  # Get the output vocabulary size.
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(source.device)  # Initialize output tensor.
        
        encoder_hidden = self.encoder(source)  # Get encoder hidden states.
        
        if isinstance(encoder_hidden, tuple):  # If encoder hidden states is a tuple (LSTM case).
            h_n, c_n = encoder_hidden  # Unpack hidden states.
            if self.encoder.bidirectional:  # If encoder is bidirectional.
                h_n = torch.cat([h_n[i:i+1] for i in range(0, h_n.shape[0], 2)] + [h_n[i:i+1] for i in range(1, h_n.shape[0], 2)], dim=2)  # Concatenate forward and backward hidden states.
                c_n = torch.cat([c_n[i:i+1] for i in range(0, c_n.shape[0], 2)] + [c_n[i:i+1] for i in range(1, c_n.shape[0], 2)], dim=2)  # Concatenate forward and backward cell states.
            
            if h_n.size(0) < self.decoder.num_layers:  # If decoder has more layers than encoder.
                # Create zero tensors with the SAME hidden dimension as h_n
                hidden_dim = h_n.size(2)  # Get the actual hidden dimension from h_n
                zero_h = torch.zeros(self.decoder.num_layers - h_n.size(0), batch_size, hidden_dim, device=h_n.device)  # Create zero tensor for hidden states.
                zero_c = torch.zeros(self.decoder.num_layers - c_n.size(0), batch_size, hidden_dim, device=c_n.device)  # Create zero tensor for cell states.
                h_n = torch.cat([h_n, zero_h], dim=0)  # Concatenate zero tensor to adjust hidden states shape.
                c_n = torch.cat([c_n, zero_c], dim=0)  # Concatenate zero tensor to adjust cell states shape.
            encoder_hidden = (h_n[:self.decoder.num_layers], c_n[:self.decoder.num_layers])  # Update encoder hidden states.
        else:  # If encoder hidden states is not a tuple (GRU/RNN case).
            h_n = encoder_hidden  # Use hidden states directly.
            if self.encoder.bidirectional:  # If encoder is bidirectional.
                h_n = torch.cat([h_n[i:i+1] for i in range(0, h_n.shape[0], 2)] + [h_n[i:i+1] for i in range(1, h_n.shape[0], 2)], dim=2)  # Concatenate forward and backward hidden states.
            if h_n.size(0) < self.decoder.num_layers:  # If decoder has more layers than encoder.
                # Create zero tensor with the SAME hidden dimension as h_n
                hidden_dim = h_n.size(2)  # Get the actual hidden dimension from h_n
                zero_h = torch.zeros(self.decoder.num_layers - h_n.size(0), batch_size, hidden_dim, device=h_n.device)  # Create zero tensor for hidden states.
                h_n = torch.cat([h_n, zero_h], dim=0)  # Concatenate zero tensor to adjust hidden states shape.
            encoder_hidden = h_n[:self.decoder.num_layers]  # Update encoder hidden states.
        
        decoder_input = target[:, 0]  # Get the decoder input for the first time step.
                    
        for t in range(1, target_len):  # Iterate over target sequence.
            decoder_output, encoder_hidden = self.decoder(decoder_input, encoder_hidden)  # Get decoder output and update hidden states.
            outputs[:, t] = decoder_output  # Store decoder output.
            teacher_force = torch.rand(1) < teaching_force_ratio  # Determine whether to use teacher forcing.
            top1 = decoder_output.argmax(1)  # Get the predicted token.
            decoder_input = target[:, t] if teacher_force else top1  # Update decoder input based on teacher forcing.
        return outputs  # Return the final output tensor.

In [ ]:
INPUT_DIM = 100  # Set the size of the input vocabulary.
OUTPUT_DIM = 100  # Set the size of the output vocabulary.
ENC_EMB_DIM = 256  # Set the dimension of the input embeddings.
DEC_EMB_DIM = 256  # Set the dimension of the output embeddings.
HID_DIM = 512  # Set the dimension of the hidden states.
ENC_LAYERS = 1  # Set the number of layers in the encoder.
DEC_LAYERS = 3  # Set the number of layers in the decoder.
ENC_RNN_CELL = 'gru'  # Specify the RNN cell type for the encoder.
DEC_RNN_CELL = 'gru'  # Specify the RNN cell type for the decoder.

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_LAYERS, ENC_RNN_CELL, dropout=0.3, bidirectional=True)  # Initialize the encoder module.
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_LAYERS, encoder.num_layers, DEC_RNN_CELL, dropout=0.3, bidirectional=True)  # Initialize the decoder module.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Choose the appropriate device for computation.
print(f"Using device: {device}")  # Print the chosen device.
model = Seq_to_Seq(encoder, decoder).to(device)  # Initialize the sequence-to-sequence model and move it to the selected device.
print(model)  # Print the model summary.

In [ ]:
# # Define a function to create a vocabulary set from a given text
# def create_vocab(text):
#     # Create a set of unique characters found in the text
#     # Each word in the text is processed to extract its characters
#     vocab = set(char for word in text for char in word)
#     # Add a padding token to the vocabulary
#     vocab.add('<pad>')
#     # Add a start-of-sequence token to the vocabulary
#     vocab.add('<sos>')  # Start of sequence token
#     # Add an end-of-sequence token to the vocabulary
#     vocab.add('<eos>')  # End of sequence token
#     # Return the complete set of vocabulary items
#     return vocab

In [ ]:
# First, let's modify the load_data function to ensure correct column ordering
def load_data(path):
    # Make sure we're correctly identifying which column is which
    df = pd.read_csv(path, header=None, names=['telugu', 'transliteration', 'score'], sep='\t')
    
    # Check a few samples to verify column ordering
    #print("Sample data:")
    #print(df.head(3))
    
    # Return the transliteration and Telugu columns in the right order
    return df['transliteration'], df['telugu']  # Note the order - transliteration first, Telugu second

In [ ]:
# Load Telugu and transliteration training data from specified path
trans_train, tel_train = load_data('/kaggle/input/dakshina-dataset/te.translit.sampled.train.tsv')

In [ ]:
print(trans_train)

In [ ]:
print(tel_train)

In [ ]:
# Define a function to create a vocabulary set from a given text
def create_vocab(text):
    # Create a set of unique characters found in the text
    # First convert all items to strings and skip any NaN values
    vocab = set()
    for word in text:
        if pd.notna(word):  # Skip NaN values
            for char in str(word):  # Convert to string to handle any non-string values
                vocab.add(char)
    
    # Add special tokens to the vocabulary
    vocab.add('<pad>')
    vocab.add('<sos>')  # Start of sequence token
    vocab.add('<eos>')  # End of sequence token
    
    # Return the complete set of vocabulary items
    return vocab

# Create a vocabulary from the transliteration training data
trans_vocab = create_vocab(trans_train)
# Create a vocabulary from the Telugu training data
tel_vocab = create_vocab(tel_train)

In [ ]:
print(len(trans_vocab))
print(trans_vocab)

In [ ]:
print(len(tel_vocab))
print(tel_vocab)

In [ ]:
# Map each token in the trans vocabulary to a unique index
trans_token_to_index = {token: index for index, token in enumerate(sorted(trans_vocab))}
# Map each token in the tel vocabulary to a unique index
tel_token_to_index = {token: index for index, token in enumerate(sorted(tel_vocab))}

In [ ]:
# Print the dictionary mapping trans tokens to indices
print(trans_token_to_index)

In [ ]:
# Print the dictionary mapping tel tokens to indices
print(tel_token_to_index)

In [ ]:
# Define a Dataset class for handling translation and telugu word pairs
class DakshinaDataset(Dataset):
    def __init__(self, trans_words, tel_words, trans_token_to_index, tel_token_to_index):
        # Store the lists of trans and tel words
        self.trans_words = trans_words
        self.tel_words = tel_words
        # Store the dictionaries that map characters to indices for both languages
        self.trans_token_to_index = trans_token_to_index
        self.tel_token_to_index = tel_token_to_index

    def __len__(self):
        # Return the number of word pairs in the dataset
        return len(self.trans_words)

    def __getitem__(self, index):
        # Fetching the trans and tel words at the specified index
        trans_word = self.trans_words.iloc[index]
#         print(trans_word)
        tel_word = self.tel_words.iloc[index]
#         print(teel_word)
        # Convert the trans word into indices using the trans_token_to_index mapping
        trans_indices = [trans_token_to_index[char] for char in trans_word]
#         print(trans_indices)
        # Convert the telugu word into indices, adding <sos> and <eos> tokens
        tel_indices = [tel_token_to_index['<sos>']] + [tel_token_to_index[char] for char in tel_word] + [tel_token_to_index['<eos>']]
#         print(tel_indices)
        # Return the indices as tensor objects
        return torch.tensor(trans_indices, dtype=torch.long), torch.tensor(tel_indices, dtype=torch.long)

In [ ]:
# Define a function for padding sequences and packing batches
# packet_fn specifies a function to control how batches are created from the individual data items
def packet_fn(batch):
    # Unzip the batch to separate trans and telugu indices
    trans, tel = zip(*batch)
#     print(trans, tel)
    # Pad the sequences of trans indices
    trans_padded = pad_sequence(trans, batch_first=True, padding_value=trans_token_to_index['<pad>'])
#     print(trans_padded)
    # Pad the sequences of tel indices
    tel_padded = pad_sequence(tel, batch_first=True, padding_value=tel_token_to_index['<pad>'])
#     print(tel_padded)
    # Return the padded batches
    return trans_padded, tel_padded

In [ ]:
# Load training data into the DakshinaDataset
train_dataset = DakshinaDataset(trans_train, tel_train, trans_token_to_index, tel_token_to_index)
# Create a DataLoader to batch and shuffle the dataset
# packet_fn specifies a function to control how batches are created from the individual data items
train_loader = DataLoader(train_dataset, batch_size = 64, collate_fn=packet_fn, shuffle=True)

In [ ]:
print(len(train_dataset))

In [ ]:
print(train_dataset[4000])

In [ ]:
# Define a word accuracy function for word-level accuracy
def word_accuracy(outputs, targets, ignore_index):
    # Assuming outputs and targets are batched sequences of token indices
    # Ignoring <pad> tokens as specified by `ignore_index`
    correct = 0  # Initialize the count of correct predictions.
    total = 0  # Initialize the total number of sequences.
    for out, tar in zip(outputs, targets):  # Iterate over each output and target pair.
        # Ignoring padding in accuracy calculation
#         print('out bef pad:',out)  # Uncomment to print the output before removing padding.
#         print('tar:',tar)  # Uncomment to print the target.
        out = out[out != ignore_index]  # Remove padding tokens from the output.
        tar = tar[tar != ignore_index]  # Remove padding tokens from the target.
        ignore_index_eos = 0  # Define an ignore index for end of sequence.
        out = out[out != ignore_index_eos]  # Remove end of sequence tokens from the output.
        tar = tar[tar != ignore_index_eos]  # Remove end of sequence tokens from the target.
#         print('out aft pad:',out)  # Uncomment to print the output after removing padding.
#         print('tar:',tar)  # Uncomment to print the target after removing padding.
        if torch.equal(out, tar):  # Check if the processed output and target are identical.
            correct += 1  # Increment the correct count.
#             print('correct:',correct)  # Uncomment to print the correct count.
        total += 1  # Increment the total count.
#         print('total:',total)  # Uncomment to print the total count.
    return correct / total if total > 0 else 0  # Calculate and return the accuracy.

In [ ]:
def train(model, iterator, optimizer, criterion, clip, device, ignore_index):
    # Set the model to training mode
    model.train()
    # Initialize epoch loss and accuracy
    epoch_loss = 0
    epoch_acc = 0
    
    # Iterate through the data iterator
    for source, target in iterator:
        # Move source and target tensors to the specified device
        source = source.to(device)
        target = target.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass: compute model predictions
        output = model(source, target)
        
        output_dim = output.shape[-1]
        # Slice the output and target tensors to remove <sos> token and maintain sequence structure
        output = output[:, 1:, :]
        target = target[:, 1:]
        
        # Flatten all dimensions except for the batch dimension for loss calculation
        output_flat = output.reshape(-1, output_dim)
        target_flat = target.reshape(-1)
        #print(output_flat.shape)
        #print(target_flat.shape)
        
        # Compute the loss
        loss = criterion(output_flat, target_flat)
        # Calculate word-by-word accuracy
        acc = word_accuracy(output.argmax(dim=2), target, ignore_index)
        
        # Backpropagation
        loss.backward()
        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        # Update model parameters
        optimizer.step()
        
        # Accumulate epoch loss and accuracy
        epoch_loss += loss.item()
        epoch_acc += acc
    
    # Return average epoch loss and accuracy
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device, ignore_index):
    # Set the model to evaluation mode
    model.eval()
    # Initialize epoch loss and accuracy
    epoch_loss = 0
    epoch_acc = 0
    
    # Iterate through the data iterator
    with torch.no_grad():
        for source, target in iterator:
            # Move source and target tensors to the specified device
            source = source.to(device)
            target = target.to(device)
            
            # Forward pass: compute model predictions without teacher forcing
            output = model(source, target, 0)
            output_dim = output.shape[-1]
            # Slice the output and target tensors to remove <sos> token and maintain sequence structure
            output = output[:, 1:, :]
            target = target[:, 1:]
            
            # Flatten all dimensions except for the batch dimension for loss calculation
            output_flat = output.reshape(-1, output_dim)
            target_flat = target.reshape(-1)
            
            # Compute the loss
            loss = criterion(output_flat, target_flat)
            # Calculate word-by-word accuracy
            acc = word_accuracy(output.argmax(dim=2), target, ignore_index)
            
            # Accumulate epoch loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc
            
    # Return average epoch loss and accuracy
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# Load validation data by reading a TSV file
trans_valid, tel_valid = load_data('/kaggle/input/dakshina-dataset/te.translit.sampled.dev.tsv')

# Create a validation dataset using the DakshinaDataset class.
valid_dataset = DakshinaDataset(trans_valid, tel_valid, trans_token_to_index, tel_token_to_index)

# Create a DataLoader to batch and shuffle the dataset
# 'collate_fn=packet_fn' specifies a function to control how batches are created from the individual data items.
# 'shuffle=True' ensures that the data is shuffled at every epoch which helps to reduce model overfitting
valid_loader = DataLoader(valid_dataset, batch_size=64, collate_fn=packet_fn, shuffle=True)

In [ ]:
print(len(trans_valid))

In [ ]:
# -embed_size-64-layers_enc-3-layers_dec-3-hid_size-512-cell_type-lstm-bidirectional-True-dropout-0.2
# Define the dimensions and configurations for the encoder and decoder
INPUT_DIM = 100
OUTPUT_DIM = 100
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
HID_DIM = 512
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_RNN_CELL = 'lstm'
DEC_RNN_CELL = 'lstm'

# Initialize the encoder with the specified parameters
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_LAYERS, ENC_RNN_CELL, dropout=0.2, bidirectional=True)
# Initialize the decoder with the specified parameters, using the number of encoder layers
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_LAYERS, encoder.num_layers, DEC_RNN_CELL, dropout=0.2, bidirectional=True)
# Determine the device for model training (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Initialize the sequence-to-sequence model with the encoder and decoder
model = Seq_to_Seq(encoder, decoder).to(device)
print(model)

In [ ]:
# Original method (causing the error)
def __getitem__(self, index):
    # Get the transliteration and Telugu words for the current index
    trans_word = self.trans_data.iloc[index]
    tel_word = self.tel_data.iloc[index]
    
    # Convert the trans word into indices using the trans_token_to_index mapping
    trans_indices = [trans_token_to_index[char] for char in trans_word]  # Error line
    
    # Convert the telugu word into indices, adding <sos> and <eos> tokens
    tel_indices = [tel_token_to_index['<sos>']] + [tel_token_to_index[char] for char in tel_word] + [tel_token_to_index['<eos>']]
    
    # Convert lists to tensors
    return torch.tensor(trans_indices), torch.tensor(tel_indices)

In [ ]:
# Redefine your Dataset class
class TransliterationDataset(torch.utils.data.Dataset):
    def __init__(self, trans_data, tel_data):
        self.trans_data = trans_data
        self.tel_data = tel_data
    
    def __len__(self):
        return len(self.trans_data)
    
    def __getitem__(self, index):
        # Get the transliteration and Telugu words for the current index
        trans_word = self.trans_data.iloc[index]
        tel_word = self.tel_data.iloc[index]
        
        # Handle NaN or float values by converting to string
        if pd.isna(trans_word) or not isinstance(trans_word, str):
            trans_word = str(trans_word) if not pd.isna(trans_word) else ""
            
        if pd.isna(tel_word) or not isinstance(tel_word, str):
            tel_word = str(tel_word) if not pd.isna(tel_word) else ""
        
        # Convert the transliteration word into indices
        trans_indices = []
        for char in trans_word:
            if char in trans_token_to_index:
                trans_indices.append(trans_token_to_index[char])
            else:
                # Skip unknown characters
                continue
        
        # If no valid characters were found, use a minimal valid sequence
        if not trans_indices:
            trans_indices = [trans_token_to_index.get('<pad>', 0)]
        
        # Convert the Telugu word into indices, adding <sos> and <eos> tokens
        tel_indices = [tel_token_to_index['<sos>']]
        for char in tel_word:
            if char in tel_token_to_index:
                tel_indices.append(tel_token_to_index[char])
            else:
                # Skip unknown characters
                continue
        tel_indices.append(tel_token_to_index['<eos>'])
        
        # Convert lists to tensors
        return torch.tensor(trans_indices), torch.tensor(tel_indices)

In [ ]:
# Load the test data from the specified TSV file location
trans_test, tel_test = load_data('/kaggle/input/dakshina-dataset/te.translit.sampled.test.tsv')

# Create test_dataset using the AksharantarDataset class, initializing it with test data
# and corresponding token-to-index mappings for both Latin and Bangla scripts
test_dataset = DakshinaDataset(trans_test, tel_test, trans_token_to_index, tel_token_to_index)

# A DataLoader for the test dataset. Here, the batch size is set to 1, indicates
# that the model will process one item at a time. This is for testing to make
# detailed predictions per sample without batching effects.
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=packet_fn, shuffle=False)
# print(test_dataset[0])

In [ ]:
# Define batch size
BATCH_SIZE = 32  # You can adjust this as needed

# Define a padding collate function to handle variable length sequences
def pad_collate_fn(batch):
    # Sort the batch in descending order by source length
    batch.sort(key=lambda x: len(x[0]), reverse=True)
    
    # Get source and target sequences
    src_seqs, trg_seqs = zip(*batch)
    
    # Find length of longest sequence in batch
    src_lengths = [len(s) for s in src_seqs]
    trg_lengths = [len(t) for t in trg_seqs]
    max_src_len = max(src_lengths)
    max_trg_len = max(trg_lengths)
    
    # Pad source sequences
    src_padded = torch.zeros(len(src_seqs), max_src_len).long()
    for i, seq in enumerate(src_seqs):
        end = src_lengths[i]
        src_padded[i, :end] = seq
    
    # Pad target sequences
    trg_padded = torch.zeros(len(trg_seqs), max_trg_len).long()
    for i, seq in enumerate(trg_seqs):
        end = trg_lengths[i]
        trg_padded[i, :end] = seq
    
    # Return padded tensors and lengths
    return src_padded, trg_padded

# Create datasets
train_dataset = TransliterationDataset(trans_train, tel_train)
valid_dataset = TransliterationDataset(trans_valid, tel_valid)
test_dataset = TransliterationDataset(trans_test, tel_test)

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    collate_fn=pad_collate_fn
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE,
    collate_fn=pad_collate_fn
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE,
    collate_fn=pad_collate_fn
)

In [ ]:
# Setting the number of epochs the training process should run
NUM_EPOCHS = 1
# Set the maximum norm of the gradients to 1 to prevent exploding gradients
CLIP = 1
# Create Adam optimizer with default parameters
optimizer = torch.optim.Adam(model.parameters())
# Padding token index should be ignored in loss calculation
ignore_index = tel_token_to_index['<pad>']
# Define the loss function with 'ignore_index' to avoid affecting loss calculation with padding tokens
criterion = nn.CrossEntropyLoss(ignore_index=ignore_index).to(device)

# Start the training process for the defined number of epochs
for epoch in range(NUM_EPOCHS):
    # Doing training on the train dataset and return average loss and accuracy
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, CLIP, device, ignore_index)
    # Evaluating the model on the validation dataset and return average loss and accuracy
    val_loss, val_accuracy = evaluate(model, valid_loader, criterion, device, ignore_index)
    
    # Print the loss and accuracy for each epoch
    print(f'Epoch: {epoch+1}')
    print(f'\tTrain_Loss: {train_loss:.3f}, Train_Accuracy: {train_accuracy*100:.2f}%')
    print(f'\tVal_Loss: {val_loss:.3f},  Val_Accuracy: {val_accuracy*100:.2f}%')

In [ ]:
print(len(trans_test))

In [ ]:
def decode_indices(indices, index_to_token):
    # Filter out indices for padding, start-of-sequence, and end-of-sequence tokens to ensure only valid character indices are decoded
    valid_indices = [index for index in indices if index in index_to_token and index not in (tel_token_to_index['<pad>'], tel_token_to_index['<sos>'], tel_token_to_index['<eos>'])]
    # Convert each index to its corresponding character and join them to form the decoded string
    return ''.join([index_to_token[index] for index in valid_indices])

In [ ]:
def predict(model, iterator, device):
    # Set the model to evaluation mode to disable dropout or batch normalization effects during inference
    model.eval()
    predictions = []
    # Disables gradient calculations for performance improvement since they are not needed in inference
    with torch.no_grad():
        for source, target in iterator:
            # Ensure the source and target tensors are on the correct device (GPU or CPU)
            source = source.to(device)
            target = target.to(device)
            # Obtain model output without teacher forcing (i.e., the model relies entirely on its predictions)
            output = model(source, target, 0)
            # Get the index with the highest probability from output predictions
            output = output.argmax(2)
            # Convert tensors to CPU numpy arrays for easier manipulation and extraction
            source = source.cpu().numpy()
            output = output.cpu().numpy()
            target = target.cpu().numpy()
            # Store the tuple of source and decoded output predictions
            predictions.append((source, target, output))
    # Return all predictions made over the iterator
    return predictions

In [ ]:
# Create dictionaries to map indices back to characters, observing the interpretation of prediction outputs
trans_index_to_token = {index: char for char, index in trans_token_to_index.items()}
tel_index_to_token = {index: char for char, index in tel_token_to_index.items()}

In [ ]:
print(trans_index_to_token)

In [ ]:
print(tel_index_to_token)

In [ ]:
# Taking the prediction function to generate outputs for all samples in the test_loader
test_predictions = predict(model, test_loader, device)
# print(len(test_predictions[0]))
# Loop through the list of tuples containing source and output indices from the test predictions
for source_indices, target_indices, output_indices in test_predictions:
    # Iterate through each example in the batch. This is necessary as batches may contain multiple examples
    for i in range(source_indices.shape[0]):
        # Decode the source indices to their corresponding text using the mapping dictionary for trans script
        input_text = decode_indices(source_indices[i], trans_index_to_token)
        
        target_text = decode_indices(target_indices[i], tel_index_to_token)

        # Decode the output indices to their corresponding text using the mapping dictionary for telugu script
        predicted_text = decode_indices(output_indices[i], tel_index_to_token)
        # Print the original input text and its corresponding predicted transliteration
        #print(f'Input Text: {input_text} -> Actual Text: {target_text} -> Predicted Text: {predicted_text}')

In [40]:
import pandas as pd

# Create lists to store the data
input_texts = []
actual_texts = []
predicted_texts = []
test_predictions = predict(model, test_loader, device)

# Loop through the list of tuples containing source and output indices from the test predictions
for source_indices, target_indices, output_indices in test_predictions:
    # Iterate through each example in the batch. This is necessary as batches may contain multiple examples
    for i in range(source_indices.shape[0]):
        # Decode the source indices to their corresponding text using the mapping dictionary for trans script
        input_text = decode_indices(source_indices[i], trans_index_to_token)
        target_text = decode_indices(target_indices[i], tel_index_to_token)
        # Decode the output indices to their corresponding text using the mapping dictionary for telugu script
        predicted_text = decode_indices(output_indices[i], tel_index_to_token)
        # Append the texts to the lists
        input_texts.append(input_text)
        actual_texts.append(target_text)
        predicted_texts.append(predicted_text)

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Input Text': input_texts,
    'Actual Text': actual_texts,
    'Predicted Text': predicted_texts
})

# Save the DataFrame to a CSV file
df.to_csv('predictions_without_attn.csv', index=False, encoding='utf-8')

In [41]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key) #key="7836c2516a56fb9071600184ea90a0f6357f99dc"

Enter your API: 7836c2516a56fb9071600184ea90a0f6357f99dc


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tentuvenkatesh2 (tentuvenkatesh2-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [42]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep all final new lr 5',
    'metric': {
        'name': 'Val_Accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'input_embed_size': {
            'values': [16,32,64,256,512]
        },
        'num_enc_layers':{
            'values': [1,2,3]
        },
        'num_dec_layers':{
            'values': [1,2,3]
        },
        'hid_layer_size': {
            'values': [16,32,64,256,512]
        },
        'cell_type': {
            'values': ['rnn','gru','lstm']
        },
        'bidirectional':{
            'values': [True, False]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'new_learning_rate':{
            'values': [0.001,0.01,0.1]
        }
#       'beam search in decoder with different beam sizes': 
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="DA6401_A3_P_1")

Create sweep with ID: lrdb1qkr
Sweep URL: https://wandb.ai/tentuvenkatesh2-indian-institute-of-technology-madras/DA6401_A3_P_1/sweeps/lrdb1qkr


In [43]:
import wandb

def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name from configuration
        run_name = "-embed_size-"+str(wandb.config.input_embed_size)+"-layers_enc-"+str(wandb.config.num_enc_layers)+"-layers_dec-"+str(wandb.config.num_dec_layers)+"-hid_size-"+str(wandb.config.hid_layer_size)+"-cell_type-"+wandb.config.cell_type+"-bidirectional-"+str(wandb.config.bidirectional)+"-dropout-"+str(wandb.config.dropout)+"-lr-"+str(wandb.config.new_learning_rate)
        wandb.run.name = run_name

        # Constants defining the dimensions of the input and output character sets
        INPUT_DIM = 100  # size of the trans character set
        OUTPUT_DIM = 100  # size of the telugu character set

        # Constants defining the dimensions of the embeddings for encoder and decoder
        ENC_EMB_DIM = wandb.config.input_embed_size  # Encoder embedding dimension
        DEC_EMB_DIM = wandb.config.input_embed_size  # Decoder embedding dimension

        # Constants defining the dimension of the hidden layers for encoder and decoder
        HID_DIM = wandb.config.hid_layer_size  # Hidden dimension size

        # Constants defining the number of layers for encoder and decoder
        ENC_LAYERS = wandb.config.num_enc_layers  # Number of layers in the encoder
        DEC_LAYERS = wandb.config.num_dec_layers  # Number of layers in the decoder
        

        # Constants defining the type of RNN cell to use for encoder and decoder
        ENC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the encoder
        DEC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the decoder

        # Instantiate the encoder with specified configurations
        encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_LAYERS, ENC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)
        # Instantiate the decoder with specified configurations
        decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_LAYERS, encoder.num_layers, DEC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)

        # Determine the computing device (CUDA if available, otherwise CPU)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Print the device will be used
        print(f"Using device: {device}")

        # Instantiate the Seq_to_Seq model and move it to the chosen computing device
        model = Seq_to_Seq(encoder, decoder).to(device)
        print(model)
        
        
        # Setting the number of epochs the training process should run
        NUM_EPOCHS = 5
        # Set the maximum norm of the gradients to 1 to prevent exploding gradients
        CLIP = 1
        # Initialize the optimizer, Adam
        optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.new_learning_rate)  # Set the learning rate to 0.001


        # Padding token index should be ignored in loss calculation
        ignore_index = tel_token_to_index['<pad>']
        # Define the loss function with 'ignore_index' to avoid affecting loss calculation with padding tokens
        criterion = nn.CrossEntropyLoss(ignore_index=ignore_index).to(device)

        # Start the training process for the defined number of epochs
        for epoch in range(NUM_EPOCHS):
            # Doing training on the train dataset and return average loss and accuracy
            train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, CLIP, device, ignore_index)
            # Evaluating the model on the validation dataset and return average loss and accuracy
            val_loss, val_accuracy = evaluate(model, valid_loader, criterion, device, ignore_index)

            # Print the loss and accuracy for each epoch
            print(f'Epoch: {epoch+1}')
            print(f'\tTrain_Loss: {train_loss:.3f}, Train_Accuracy: {train_accuracy*100:.2f}%')
            print(f'\tVal_Loss: {val_loss:.3f},  Val_Accuracy: {val_accuracy*100:.2f}%')
            wandb.log({"Epoch": {epoch+1}})
            wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": train_loss})
            wandb.log({"Val_Accuracy": val_accuracy * 100, "Val_Loss": val_loss})


wandb.agent(sweep_id, function=main, count=50)
wandb.finish()

wandb: Agent Starting Run: ddvoksm8 with config:
wandb: 	bidirectional: True
wandb: 	cell_type: rnn
wandb: 	dropout: 0.2
wandb: 	hid_layer_size: 16
wandb: 	input_embed_size: 16
wandb: 	new_learning_rate: 0.001
wandb: 	num_dec_layers: 3
wandb: 	num_enc_layers: 3


Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 16)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): RNN(16, 8, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 16)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): RNN(16, 48, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=48, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 1.814, Train_Accuracy: 0.01%
	Val_Loss: 2.055,  Val_Accuracy: 0.00%
Epoch: 2
	Train_Loss: 1.606, Train_Accuracy: 0.02%
	Val_Loss: 1.961,  Val_Accuracy: 0.00%
Epoch: 3
	Train_Loss: 1.548, Train_Accuracy: 0.02%
	Val_Loss: 1.862,  Val_Accuracy: 0.00%
Epoch: 4
	Train_Loss: 1.505, Train_Accuracy: 0.02%
	Val_Loss: 1.812,  Val_Accuracy: 0.00%
Epoch: 5
	Train_Loss: 1.473, Train_Accuracy: 0.03%
	Val_Loss: 1.771,  Val_Accuracy: 0.02%


Val_Accuracy,▁▁▁▁█
Val_Loss,█▆▃▂▁
train_accuracy,▁▅▅▆█
training_loss,█▄▃▂▁
Val_Accuracy,0.01756
Val_Loss,1.77061
train_accuracy,0.02561
training_loss,1.47277


wandb: Agent Starting Run: ew477ivf with config:
wandb: 	bidirectional: False
wandb: 	cell_type: gru
wandb: 	dropout: 0.2
wandb: 	hid_layer_size: 32
wandb: 	input_embed_size: 16
wandb: 	new_learning_rate: 0.001
wandb: 	num_dec_layers: 1
wandb: 	num_enc_layers: 1


Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 16)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(16, 32, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 16)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): GRU(16, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 1.881, Train_Accuracy: 0.00%
	Val_Loss: 2.063,  Val_Accuracy: 0.00%
Epoch: 2
	Train_Loss: 1.598, Train_Accuracy: 0.02%
	Val_Loss: 1.891,  Val_Accuracy: 0.02%
Epoch: 3
	Train_Loss: 1.492, Train_Accuracy: 0.06%
	Val_Loss: 1.773,  Val_Accuracy: 0.11%
Epoch: 4
	Train_Loss: 1.408, Train_Accuracy: 0.12%
	Val_Loss: 1.668,  Val_Accuracy: 0.21%
Epoch: 5
	Train_Loss: 1.332, Train_Accuracy: 0.19%
	Val_Loss: 1.564,  Val_Accuracy: 0.53%


...

Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 512)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(512, 512, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 512)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=512, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 1.036, Train_Accuracy: 5.12%
	Val_Loss: 0.841,  Val_Accuracy: 19.62%
Epoch: 2
	Train_Loss: 0.497, Train_Accuracy: 23.00%
	Val_Loss: 0.711,  Val_Accuracy: 33.38%
Epoch: 3
	Train_Loss: 0.379, Train_Accuracy: 33.58%
	Val_Loss: 0.622,  Val_Accuracy: 38.72%
Epoch: 4
	Train_Loss: 0.314, Train_Accuracy: 41.14%
	Val_Loss: 0.611,  Val_Accuracy: 41.72%
Epoch: 5
	Train_Loss: 0.278, Train_Accuracy: 45.67%
	Val_Loss: 0.565,  Val_Accuracy: 43.77%


Val_Accuracy,▁▅▇▇█
Val_Loss,█▅▂▂▁
train_accuracy,▁▄▆▇█
training_loss,█▃▂▁▁
Val_Accuracy,43.76848
Val_Loss,0.56502
train_accuracy,45.66878
training_loss,0.27811


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ekphg063 with config:
wandb: 	bidirectional: False
wandb: 	cell_type: lstm
wandb: 	dropout: 0.2
wandb: 	hid_layer_size: 512
wandb: 	input_embed_size: 512
wandb: 	new_learning_rate: 0.001
wandb: 	num_dec_layers: 3
wandb: 	num_enc_layers: 1


Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 512)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(512, 512, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 512)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(512, 512, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=512, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 1.178, Train_Accuracy: 2.64%
	Val_Loss: 0.997,  Val_Accuracy: 13.81%
Epoch: 2
	Train_Loss: 0.523, Train_Accuracy: 21.05%
	Val_Loss: 0.704,  Val_Accuracy: 31.05%
Epoch: 3
	Train_Loss: 0.366, Train_Accuracy: 35.17%
	Val_Loss: 0.627,  Val_Accuracy: 37.68%
Epoch: 4
	Train_Loss: 0.291, Train_Accuracy: 43.94%
	Val_Loss: 0.615,  Val_Accuracy: 40.20%
Epoch: 5
	Train_Loss: 0.246, Train_Accuracy: 50.09%
	Val_Loss: 0.624,  Val_Accuracy: 42.67%


Val_Accuracy,▁▅▇▇█
Val_Loss,█▃▁▁▁
train_accuracy,▁▄▆▇█
training_loss,█▃▂▁▁
Val_Accuracy,42.67445
Val_Loss,0.62396
train_accuracy,50.09221
training_loss,0.24595


In [44]:
import pandas as pd
df = pd.read_csv("predictions_without_attn.csv")
df.head()

,Input Text,Actual Text,Predicted Text
0,amgiikarimchaadu,అంగీకరించాడు,అంగికరించాడు
1,angeekarinchaadu,అంగీకరించాడు,అంగీకరించాడు
2,angiikarinchaadu,అంగీకరించాడు,అంగికరించాడు
3,angeekarinchaali,అంగీకరించాలి,అంగీకరించాలి
4,angeekarinchaka,అంగీకరించక,అంజీకరించక


In [ ]:
# -embed_size-64-layers_enc-2-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	56.801633648728554
# -embed_size-64-layers_enc-1-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-True-dropout-0.3-lr-0.001	55.60134535777647
# -embed_size-64-layers_enc-2-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-True-dropout-0.3-lr-0.001	55.17445298639858
# -embed_size-256-layers_enc-3-layers_dec-3-hid_size-512-cell_type-lstm-bidirectional-True-dropout-0.3-lr-0.001	54.30865612063867
# -embed_size-32-layers_enc-3-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	53.88083973979894
# -embed_size-32-layers_enc-1-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	53.20169278533412
# -embed_size-512-layers_enc-2-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-True-dropout-0.3-lr-0.001	52.96237433471319
# -embed_size-256-layers_enc-1-layers_dec-3-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	47.625295683027794
# -embed_size-512-layers_enc-1-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	43.76848018923714
# -embed_size-512-layers_enc-1-layers_dec-3-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.2-lr-0.001	42.674452986398585

In [48]:
# -embed_size-64-layers_enc-2-layers_dec-2-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	56.801633648728554
sweep_config = {
    'method': 'bayes',
    'name' : 'Best sweep test',
    'metric': {
        'name': 'Test_Accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'input_embed_size': {
            'values': [64]
        },
        'num_enc_layers':{
            'values': [2]
        },
        'num_dec_layers':{
            'values': [2]
        },
        'hid_layer_size': {
            'values': [512]
        },
        'cell_type': {
            'values': ['lstm']
        },
        'bidirectional':{
            'values': [False]
        },
        'dropout': {
            'values': [0.3]
        },
        'new_learning_rate':{
            'values': [0.001]
        }
#       'beam search in decoder with different beam sizes': 
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="DA6401_A3_P_1_Best")

Create sweep with ID: oei6a9zj
Sweep URL: https://wandb.ai/tentuvenkatesh2-indian-institute-of-technology-madras/DA6401_A3_P_1_Best/sweeps/oei6a9zj


In [49]:
import wandb

def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name from configuration
        run_name = "-embed_size-"+str(wandb.config.input_embed_size)+"-layers_enc-"+str(wandb.config.num_enc_layers)+"-layers_dec-"+str(wandb.config.num_dec_layers)+"-hid_size-"+str(wandb.config.hid_layer_size)+"-cell_type-"+wandb.config.cell_type+"-bidirectional-"+str(wandb.config.bidirectional)+"-dropout-"+str(wandb.config.dropout)+"-lr-"+str(wandb.config.new_learning_rate)
        wandb.run.name = run_name

        # Constants defining the dimensions of the input and output character sets
        INPUT_DIM = 100  # size of the trans character set
        OUTPUT_DIM = 100  # size of the telugu character set

        # Constants defining the dimensions of the embeddings for encoder and decoder
        ENC_EMB_DIM = wandb.config.input_embed_size  # Encoder embedding dimension
        DEC_EMB_DIM = wandb.config.input_embed_size  # Decoder embedding dimension

        # Constants defining the dimension of the hidden layers for encoder and decoder
        HID_DIM = wandb.config.hid_layer_size  # Hidden dimension size

        # Constants defining the number of layers for encoder and decoder
        ENC_LAYERS = wandb.config.num_enc_layers  # Number of layers in the encoder
        DEC_LAYERS = wandb.config.num_dec_layers  # Number of layers in the decoder
        

        # Constants defining the type of RNN cell to use for encoder and decoder
        ENC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the encoder
        DEC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the decoder

        # Instantiate the encoder with specified configurations
        encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_LAYERS, ENC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)
        # Instantiate the decoder with specified configurations
        decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_LAYERS, encoder.num_layers, DEC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)

        # Determine the computing device (CUDA if available, otherwise CPU)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Print the device will be used
        print(f"Using device: {device}")

        # Instantiate the Seq_to_Seq model and move it to the chosen computing device
        model = Seq_to_Seq(encoder, decoder).to(device)
        print(model)
        
        
        # Setting the number of epochs the training process should run
        NUM_EPOCHS = 10
        # Set the maximum norm of the gradients to 1 to prevent exploding gradients
        CLIP = 1
        # Initialize the optimizer, Adam
        optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.new_learning_rate)  # Set the learning rate to 0.001


        # Padding token index should be ignored in loss calculation
        ignore_index = tel_token_to_index['<pad>']
        # Define the loss function with 'ignore_index' to avoid affecting loss calculation with padding tokens
        criterion = nn.CrossEntropyLoss(ignore_index=ignore_index).to(device)

        # Start the training process for the defined number of epochs
        for epoch in range(NUM_EPOCHS):
            # Doing training on the train dataset and return average loss and accuracy
            train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, CLIP, device, ignore_index)
            # Evaluating the model on the validation dataset and return average loss and accuracy
            val_loss, val_accuracy = evaluate(model, test_loader, criterion, device, ignore_index)

            # Print the loss and accuracy for each epoch
            print(f'Epoch: {epoch+1}')
            print(f'\tTrain_Loss: {train_loss:.3f}, Train_Accuracy: {train_accuracy*100:.2f}%')
            print(f'\tTest_Loss: {val_loss:.3f},  Test_Accuracy: {val_accuracy*100:.2f}%')
            wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": train_loss})
            wandb.log({"Test_Accuracy": val_accuracy * 100, "Test_Loss": val_loss})


wandb.agent(sweep_id, function=main, count=1)
wandb.finish()

wandb: Agent Starting Run: 79jdhrlt with config:
wandb: 	bidirectional: False
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	hid_layer_size: 512
wandb: 	input_embed_size: 64
wandb: 	new_learning_rate: 0.001
wandb: 	num_dec_layers: 2
wandb: 	num_enc_layers: 2


Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(64, 512, num_layers=2, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(64, 512, num_layers=2, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=512, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 0.979, Train_Accuracy: 11.35%
	Test_Loss: 0.592,  Test_Accuracy: 38.90%
Epoch: 2
	Train_Loss: 0.294, Train_Accuracy: 43.49%
	Test_Loss: 0.490,  Test_Accuracy: 48.16%
Epoch: 3
	Train_Loss: 0.206, Train_Accuracy: 56.07%
	Test_Loss: 0.458,  Test_Accuracy: 53.45%
Epoch: 4
	Train_Loss: 0.162, Train_Accuracy: 63.11%
	Test_Loss: 0.479,  Test_Accuracy: 54.48%
Epoch: 5
	Train_Loss: 0.133, Train_Accuracy: 67.56%
	Test_Loss: 0.497,  Test_Accuracy: 54.95%
Epoch: 6
	Train_Loss: 0.115, Train_Accuracy: 71.25%
	Test_Loss: 0.483,  Test_Acc

Test_Accuracy,▁▅▇▇▇█████
Test_Loss,█▃▁▂▃▂▃▃▄▅
train_accuracy,▁▄▆▆▇▇▇███
training_loss,█▃▂▂▁▁▁▁▁▁
Test_Accuracy,57.00475
Test_Loss,0.52649
train_accuracy,79.27394
training_loss,0.07534


In [45]:
#-embed_size-64-layers_enc-3-layers_dec-3-hid_size-512-cell_type-lstm-bidirectional-False-dropout-0.3-lr-0.001	56.3507170313424
sweep_config = {
    'method': 'bayes',
    'name' : 'Best sweep test',
    'metric': {
        'name': 'Test_Accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'input_embed_size': {
            'values': [64]
        },
        'num_enc_layers':{
            'values': [3]
        },
        'num_dec_layers':{
            'values': [3]
        },
        'hid_layer_size': {
            'values': [512]
        },
        'cell_type': {
            'values': ['lstm']
        },
        'bidirectional':{
            'values': [False]
        },
        'dropout': {
            'values': [0.3]
        },
        'new_learning_rate':{
            'values': [0.001]
        }
#       'beam search in decoder with different beam sizes': 
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="DA6401_A3_P_1_Best")

Create sweep with ID: 70ucjzem
Sweep URL: https://wandb.ai/tentuvenkatesh2-indian-institute-of-technology-madras/DA6401_A3_P_1_Best/sweeps/70ucjzem


In [46]:
import wandb

def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name from configuration
        run_name = "-embed_size-"+str(wandb.config.input_embed_size)+"-layers_enc-"+str(wandb.config.num_enc_layers)+"-layers_dec-"+str(wandb.config.num_dec_layers)+"-hid_size-"+str(wandb.config.hid_layer_size)+"-cell_type-"+wandb.config.cell_type+"-bidirectional-"+str(wandb.config.bidirectional)+"-dropout-"+str(wandb.config.dropout)+"-lr-"+str(wandb.config.new_learning_rate)
        wandb.run.name = run_name

        # Constants defining the dimensions of the input and output character sets
        INPUT_DIM = 100  # size of the trans character set
        OUTPUT_DIM = 100  # size of the telugu character set

        # Constants defining the dimensions of the embeddings for encoder and decoder
        ENC_EMB_DIM = wandb.config.input_embed_size  # Encoder embedding dimension
        DEC_EMB_DIM = wandb.config.input_embed_size  # Decoder embedding dimension

        # Constants defining the dimension of the hidden layers for encoder and decoder
        HID_DIM = wandb.config.hid_layer_size  # Hidden dimension size

        # Constants defining the number of layers for encoder and decoder
        ENC_LAYERS = wandb.config.num_enc_layers  # Number of layers in the encoder
        DEC_LAYERS = wandb.config.num_dec_layers  # Number of layers in the decoder
        

        # Constants defining the type of RNN cell to use for encoder and decoder
        ENC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the encoder
        DEC_RNN_CELL = wandb.config.cell_type  # RNN cell type for the decoder

        # Instantiate the encoder with specified configurations
        encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_LAYERS, ENC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)
        # Instantiate the decoder with specified configurations
        decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_LAYERS, encoder.num_layers, DEC_RNN_CELL, dropout = wandb.config.dropout, bidirectional = wandb.config.bidirectional)

        # Determine the computing device (CUDA if available, otherwise CPU)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Print the device will be used
        print(f"Using device: {device}")

        # Instantiate the Seq_to_Seq model and move it to the chosen computing device
        model = Seq_to_Seq(encoder, decoder).to(device)
        print(model)
        
        
        # Setting the number of epochs the training process should run
        NUM_EPOCHS = 10
        # Set the maximum norm of the gradients to 1 to prevent exploding gradients
        CLIP = 1
        # Initialize the optimizer, Adam
        optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.new_learning_rate)  # Set the learning rate to 0.001


        # Padding token index should be ignored in loss calculation
        ignore_index = tel_token_to_index['<pad>']
        # Define the loss function with 'ignore_index' to avoid affecting loss calculation with padding tokens
        criterion = nn.CrossEntropyLoss(ignore_index=ignore_index).to(device)

        # Start the training process for the defined number of epochs
        for epoch in range(NUM_EPOCHS):
            # Doing training on the train dataset and return average loss and accuracy
            train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, CLIP, device, ignore_index)
            # Evaluating the model on the validation dataset and return average loss and accuracy
            val_loss, val_accuracy = evaluate(model, test_loader, criterion, device, ignore_index)

            # Print the loss and accuracy for each epoch
            print(f'Epoch: {epoch+1}')
            print(f'\tTrain_Loss: {train_loss:.3f}, Train_Accuracy: {train_accuracy*100:.2f}%')
            print(f'\tTest_Loss: {val_loss:.3f},  Test_Accuracy: {val_accuracy*100:.2f}%')
            wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": train_loss})
            wandb.log({"Test_Accuracy": val_accuracy * 100, "Test_Loss": val_loss})


wandb.agent(sweep_id, function=main, count=1)
wandb.finish()

wandb: Agent Starting Run: 3xx8xu84 with config:
wandb: 	bidirectional: False
wandb: 	cell_type: lstm
wandb: 	dropout: 0.3
wandb: 	hid_layer_size: 512
wandb: 	input_embed_size: 64
wandb: 	new_learning_rate: 0.001
wandb: 	num_dec_layers: 3
wandb: 	num_enc_layers: 3


Using device: cuda
Seq_to_Seq(
  (encoder): Encoder(
    (embedding): Embedding(100, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(64, 512, num_layers=3, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(100, 64)
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): LSTM(64, 512, num_layers=3, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=512, out_features=100, bias=True)
  )
)
Epoch: 1
	Train_Loss: 1.255, Train_Accuracy: 2.45%
	Test_Loss: 0.880,  Test_Accuracy: 17.77%
Epoch: 2
	Train_Loss: 0.430, Train_Accuracy: 29.68%
	Test_Loss: 0.537,  Test_Accuracy: 43.42%
Epoch: 3
	Train_Loss: 0.263, Train_Accuracy: 47.70%
	Test_Loss: 0.523,  Test_Accuracy: 48.89%
Epoch: 4
	Train_Loss: 0.202, Train_Accuracy: 56.62%
	Test_Loss: 0.485,  Test_Accuracy: 52.51%
Epoch: 5
	Train_Loss: 0.165, Train_Accuracy: 62.19%
	Test_Loss: 0.488,  Test_Accuracy: 52.97%
Epoch: 6
	Train_Loss: 0.140, Train_Accuracy: 66.61%
	Test_Loss: 0.466,  Test_Accu

Test_Accuracy,▁▆▆▇▇█████
Test_Loss,█▂▂▁▁▁▁▁▁▂
train_accuracy,▁▄▅▆▇▇▇███
training_loss,█▃▂▂▁▁▁▁▁▁
Test_Accuracy,56.67489
Test_Loss,0.535
train_accuracy,76.04974
training_loss,0.09004


In [1]:
import pandas as pd
df = pd.read_csv("predictions_without_attn.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'predictions_without_attn.csv'